### Imporrint necessary libraries

In [1]:
import os

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib as plt

import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
import tensorflow_text

from langdetect import detect


### Loading The Data

In [2]:
path_to_train_data="../Dataset/train_data.csv"
path_to_test_data="../Dataset/test_data.csv"
train_data= pd.read_csv(path_to_train_data)
test_data= pd.read_csv(path_to_test_data)
train_data.head()

,campaign_id,comment_id,comment_description,sentiment
0,2212,17908351952371091,لخسارة الوزن الزائد والكرش بمدة قياسية مع عدم ...,Negative
1,2217,17935944230085744,🔥🔥🔥,Positive
2,2215S,17899518356507020,This is so good😍 would be great it If you add ...,Negative
3,2214,18014766136389857,😍,Positive
4,2203,17924318627206870,طبق رائع ومميز تبارك الرحمن تسلم ايدك يارب 😍,Positive


- dataset caracteristics

In [3]:
print(train_data.dtypes)
train_data.groupby('sentiment').describe()

campaign_id            object
comment_id              int64
comment_description    object
sentiment              object
dtype: object


comment_id                                                          \
                count          mean           std           min           25%   
sentiment                                                                       
Irrelevant        2.0  1.803449e+16  2.727242e+13  1.801520e+16  1.802484e+16   
Negative       1082.0  1.799547e+16  1.215232e+14  1.784320e+16  1.791969e+16   
Positive       4416.0  1.799213e+16  1.177287e+14  1.784217e+16  1.791955e+16   

                                                      
                     50%           75%           max  
sentiment                                             
Irrelevant  1.803449e+16  1.804413e+16  1.805377e+16  
Negative    1.795389e+16  1.802626e+16  1.840129e+16  
Positive    1.795206e+16  1.801864e+16  1.840678e+16

### Loading and building Bert Model

In [29]:
model_path= "./Bert_model/bert_cased"
preprocessor_path="./Bert_model/bert_multi_cased_preprocessor"

def define_model(preprocessor_path,model_path):
    # Loading model and preprocessor
    input_text = Input(shape=(), dtype=tf.string, name="input_text")
    preprocessor_layer=hub.KerasLayer(preprocessor_path,name='preprocessor')
    preprocessed_inputs=preprocessor_layer(input_text)
    
    input_word_ids = preprocessed_inputs['input_word_ids']
    input_mask = preprocessed_inputs['input_mask']
    input_type_ids = preprocessed_inputs['input_type_ids']
    
    bert_layer = hub.KerasLayer(model_path, trainable=True,name='Bert_encoder')
    bert_outputs = bert_layer([input_word_ids,input_mask,input_type_ids])
    dense = Dense(256, activation='relu')(bert_outputs[0])
    output_layer = Dense(1, activation='sigmoid',name='classifier')(dense)  # Binary classification, change units for multi-class
    model = Model(input_text, output_layer)
    return model,preprocessor_layer
bert_classifier,bert_preprocessor=define_model(preprocessor_path,model_path)
bert_classifier.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_text (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 preprocessor (KerasLayer)      {'input_mask': (Non  0           ['input_text[0][0]']             
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

### Preprocessing Data

- Removing non arabic and non english comment from the dataset

In [5]:
x# Remove sentiments with irrelevant class because they are only two
irrelevant_indexes=train_data[train_data['sentiment']=='Irrelevant'].index
train_data.drop(irrelevant_indexes,inplace=True)
# remove non english and non arabic comments 
i=0
while i <= len(train_data):
    try:
        lang= detect(train_data.iloc[i,2])
        if lang !='ar' and lang !='eng':
            train_data.drop(i,inplace=True)
            print('row deleted',i)
            i=i+1
        else:
            i=i+1
            print(train_data.iloc[i,2])
            continue
    except:
        i=i+1
        print(train_data.iloc[i,2])
        continue
train_data.groupby('sentiment').describe()

🔥🔥🔥
This is so good😍 would be great it If you add subtitles to your videos. Love watching your cooks!
row deleted 2
@zainab.aleqabi مفعل هذا التطبيق بالعراق ؟
@muhamyat_alfreej
row deleted 5
❤️❤️❤️❤️❤️❤️❤️
يارب ❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️
row deleted 8
صحة و عافية😍😍
من أحلى الماركات 😍 حبيت المسابقة
👏👏👏👏👏😍😍🌟🌟👍🏽👍🏽😘😘😘🌠🌠
👌🏻👌🏻
😍روعه
row deleted 14
row deleted 15
احبها… اعملها بس من دون زبدة الفول السوداني، احط بدالها زبدة اللوز او الكاجو 😍
يم يمي ع الطعاااااامه و اللذااااااااذه 😍😍😍
بشرتك فيها حبوب، كلف، تصبغات، تبي لونها يكون موحد، مشكله المسامات الواسعه والرؤوس السودا زاعجتك.. حياك بحسابي
ياليت لو كان الفديو بدون موسيقى حبيبتي انتبهي🤍
سؤال اخت بشاير هل مسموح لكن الاعلان عن متاجر غير مسجله في معروف ؟؟؟؟
يمي 😍 شكلها لذيذ🔥
روعه الوصفه بسيطه ولذيذه 😋
صحتين ع قلبك 🙌👏😍
ياتي بها الله
تهبلين ماشاء الله تبارك الرحمن 🤍🤍🤍🤍
تسلمي يا رب يا ديمة😍❤️❤️❤️❤️❤️
پیشگویی اینده شما در این صفحه @jakoup_bertonn
row deleted 28
حلوه المسابقة 😍😍👏👏
ماشاء الله الله يحفظهم لك يارب احلى من يعلن
سالمون بصوص 

comment_id                                                          \
               count          mean           std           min           25%   
sentiment                                                                      
Negative       832.0  1.799418e+16  1.191452e+14  1.784320e+16  1.792265e+16   
Positive      3479.0  1.799269e+16  1.182644e+14  1.784243e+16  1.791960e+16   

                                                     
                    50%           75%           max  
sentiment                                            
Negative   1.795389e+16  1.802248e+16  1.840129e+16  
Positive   1.795221e+16  1.801850e+16  1.840678e+16

- Cleaning Strings

In [ ]:
train_comments=train_data['comment_description']
train_labels=train_data['sentiment']

### Evaluate pretrained Bert model

### Fine tune bert model

- Model compiling and Hyperparameters congiguration

In [ ]:
epochs=10
batch_size=10
step_per_epoch=10

metrics=[
  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
  tf.keras.metrics.Precision(name='precision')
]
bert_classifier.compile(optimizer="adam",loss='BinaryCrossentropy', metrics=metrics)

- Training the model

In [34]:
# model fit on the data
#bert_classifier.fit(
    #train_comments,
    #train_labels,
    #epochs=epochs,
    #batch_size=batch_size
#)

### Evaluate fine tuned model

### Export the model